In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# 读取数据集
data_df = pd.read_csv('./ml-100k/u.data', header=None, index_col=None)
data = data_df.values

In [3]:
# 划分数据集
def split_data(data, M, k, seed):
    test = []
    train = []
    random.seed(seed)
    for d in data:
        if random.randint(0, M) == k:
            test.append(d)
        else:
            train.append(d)
    return train, test

# 分为7：1
train_df, test_df = split_data(data, 8, 2, 2)

In [4]:
# 整理数据集   943 users on 1682 movies.
# 矩阵：横坐标表示对应一个用户给一个电影打的分数， 纵坐标表示用户id   
# train  
train_data = np.zeros((944, 1683), dtype=np.int)   
for data in train_df:
    infor = data[0].split('\t')
    user = int(infor[0])
    movie = int(infor[1])
    train_data[user][movie] = int(infor[2])
# print(train_data[2])
# print(train_data.shape)
# test
test_data = np.zeros((944, 1683))   
for data in test_df:
    infor = data[0].split('\t')
    user = int(infor[0])
    movie = int(infor[1])
    test_data[user][movie] = int(infor[2])

In [ ]:
def sgd(data_matrix, k, alpha, lam, max_cycles):
    """使用梯度下降法进行矩阵分解。

    Args:
    - data_matrix: mat, 用户物品矩阵
    - k: int, 分解矩阵的参数
    - alpha: float, 学习率
    - lam: float, 正则化参数
    - max_cycles: int, 最大迭代次数

    Returns:
    p,q: mat, 分解后的矩阵
    """
    m, n = np.shape(data_matrix)
    # initiate p & q
    p = np.mat(np.random.random((m, k)))
    q = np.mat(np.random.random((k, n)))

    # start training
    for step in range(max_cycles):
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = data_matrix[i, j]
                    for r in range(k):
                        error = error - p[i, r] * q[r, j]
                    for r in range(k):
                        p[i, r] = p[i, r] + alpha * (2 * error * q[r, j] - lam * p[i, r])
                        q[r, j] = q[r, j] + alpha * (2 * error * p[i, r] - lam * q[r, j])

        loss = 0.0
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = 0.0
                    for r in range(k):
                        error = error + p[i, r] * q[r, j]
                    # calculate loss function
                    loss = (data_matrix[i, j] - error) * (data_matrix[i, j] - error)
                    for r in range(k):
                        loss = loss + lam * (p[i, r] * p[i, r] + q[r, j] * q[r, j]) / 2

        if loss < 0.001:
            break
        if step % 1000 == 0:
            print("\titer: %d, loss: %f" % (step, loss))
    return p, q


print(sgd(train_data, 10, 0.5, 0.5, 10000))

/home/nilu/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in double_scalars
/home/nilu/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in double_scalars
/home/nilu/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/home/nilu/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in double_scalars
/home/nilu/venv_jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


	iter: 0, loss: nan
